In [27]:
### Benjamin Tollison ###
import sympy as sp
from numba import jit
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy.solvers.pde import pdsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate, Function, Eq
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)

In [3]:
rho = 0.002377 # slug/ft^3
V_v = 500 # ft^2
W = 20500 # lb
I_y_cg = 55814 # slug-ft^2
S_w = 300 # ft^2
c_w = 11.32 # ft
x_ref = .35*c_w
x_cg = x_ref

q_infty = 0.5*rho*V_v**2
Q_hat = lambda Q: Q / (2*V_v/c_w)
C_L = lambda alpha,delta_E,Q_hat: 0.14 + .42*alpha - 4.8*alpha**2 + 0.44*delta_E + (30.5 + 41.3*alpha)*Q_hat
C_M_Ref = lambda alpha,delta_E,Q_hat: -0.02 + 0.04*alpha - 0.6*delta_E - 0.08*alpha*delta_E + 0.8*delta_E**2 + 0.5*delta_E*alpha**2 - Q_hat*(5.16+3.55*alpha + 35.9*alpha**2)

d_alpha_C_L = lambda alpha,delta_E,Q_hat: .42 - 2*4.8*alpha + 41.3*Q_hat
d_delta_E_C_L = lambda alpha,delta_E,Q_hat: 0.44
d_Q_C_L = lambda alpha,delta_E,Q_hat:  30.5 + 41.3*alpha
d_alpha_C_M_Ref = lambda alpha,delta_E,Q_hat: 0.04 - 0.08*delta_E + delta_E*alpha - Q_hat*(3.55 + 2*35.9*alpha)
d_delta_E_C_M_Ref = lambda alpha,delta_E,Q_hat: -0.6 - 0.08*alpha + 1.6*delta_E* + 0.5*alpha**2
d_Q_C_M_Ref = lambda alpha,delta_E,Q_hat: -1*(5.16+3.55*alpha + 35.9*alpha**2)
# task ii
C_L_equalibrium = lambda alpha,delta_E,Q_hat: 0.14 + .42*alpha - 4.8*alpha**2 + 0.44*delta_E + (30.5 + 41.3*alpha)*Q_hat - W/(S_w*q_infty)
guess = [0,0]
equations_equal = [W/(q_infty*S_w),0,0]
tolerence = 1e-4
coeffiecent_function_list = lambda variable_list: [C_L_equalibrium(variable_list[0],variable_list[1],0),C_M_Ref(variable_list[0],variable_list[1],0)]
coeffiecent_prime_list = lambda variable_list: [d_alpha_C_L(variable_list[0],variable_list[1],0)+d_delta_E_C_L(variable_list[0],variable_list[1],0),d_alpha_C_M_Ref(variable_list[0],variable_list[1],0)+d_delta_E_C_M_Ref(variable_list[0],variable_list[1],0)]
alpha_0,delta_E0 = scipy.optimize.newton(func=coeffiecent_function_list,x0=guess,maxiter=100000,tol=1e-6)# ,fprime=coeffiecent_prime_list)
print(alpha_0,delta_E0)

0.022069875745242603 -0.028060161376896778


In [4]:
# task iii
# linear approximation
trim_C_L = W/(S_w*q_infty)
A = np.array([[4.2,0.44],[0.04,-0.6]])
A1 = np.array([[trim_C_L-0.14,0.44],[0.02,-0.6]])
A2 = np.array([[4.2,trim_C_L-0.14],[0.04,0.02]])
alpha_0_linear = np.linalg.det(A1) / np.linalg.det(A)
delta_E0_linear = np.linalg.det(A2) / np.linalg.det(A)
alpha_0_symbol,delta_E0_symbol = symbols('\\alpha_0 \\delta_{E_0}')
displayEquations(alpha_0_symbol,alpha_0_linear)
displayEquations(delta_E0_symbol,delta_E0_linear)
print('C_L should be really close to {}'.format(trim_C_L))
print('C_L = {}'.format(C_L(alpha_0,delta_E0,0)))
print('C_M = {}'.format(C_M_Ref(alpha_0,delta_E0,0)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

C_L should be really close to 0.22998176973776469
C_L = 0.13458489561319717
C_M = -0.0016085010403557624


In [25]:
d_alpha_Lift = lambda alpha,delta_E,Q: q_infty*S_w*d_alpha_C_L(alpha,delta_E,Q)
d_delta_E_Lift = lambda alpha,delta_E,Q: q_infty*S_w*d_delta_E_C_L(alpha,delta_E,Q)
d_Q_Lift = lambda alpha,delta_E,Q: q_infty*S_w*d_Q_C_L(alpha,delta_E,Q)

d_alpha_M_cg = lambda alpha,delta_E,Q: q_infty*S_w*c_w*(d_alpha_C_M_Ref(alpha,delta_E,Q) - d_alpha_C_L(alpha,delta_E,Q)*(x_ref-x_cg)/c_w)
d_delta_E_M_cg = lambda alpha,delta_E,Q: q_infty*S_w*c_w*(d_delta_E_C_M_Ref(alpha,delta_E,Q) - d_alpha_C_L(alpha,delta_E,Q)*(x_ref-x_cg)/c_w)
d_Q_M_cg = lambda alpha,delta_E,Q: q_infty*S_w*c_w*(d_Q_C_M_Ref(alpha,delta_E,Q) - d_Q_C_L(alpha,delta_E,Q)*(x_ref-x_cg)/c_w)

def JacobianMatrix(alpha,delta_E,Q):
  dal = d_alpha_Lift(alpha,delta_E,Q) / (S_w*q_infty)
  dql = d_Q_Lift(alpha,delta_E,Q) / (S_w*q_infty)
  dam = d_alpha_M_cg(alpha,delta_E,Q) / I_y_cg
  dqm = d_Q_M_cg(alpha,delta_E,Q) / I_y_cg
  return np.array([[-dal,0,1-dql],[0,0,1],[dam,0,dqm]])
display(scipy.linalg.eig(JacobianMatrix(alpha_0,delta_E0,0)))

# task vi
stablity_of_cord = {symbols('\\alpha_0'):[],symbols('\\delta_E0'):[],symbols('x_{cg}'):[],'stability':[]}
for cord_percentage in np.linspace(.20,.50,100):
  x_cg = cord_percentage*c_w
  coeffiecent_function_list = lambda variable_list: [C_L_equalibrium(variable_list[0],variable_list[1],variable_list[2]),C_M_Ref(variable_list[0],variable_list[1],variable_list[2]),Q_hat(variable_list[2])]
  alpha_0,delta_E0,Q_0 = scipy.optimize.newton(func=coeffiecent_function_list,x0=[0,0,0],maxiter=10000,tol=1e-4)# ,fprime=coeffiecent_prime_list)
  stablity_of_cord[symbols('\\alpha_0')].append(alpha_0)
  stablity_of_cord[symbols('\\delta_E0')].append(delta_E0)
  stablity_of_cord[symbols('x_{cg}')].append(x_cg)
  dam = q_infty*S_w*c_w*(d_alpha_C_M_Ref(alpha_0,delta_E0,Q_0) - d_alpha_C_L(alpha_0,delta_E0,Q_0)*(x_ref-x_cg)/c_w)
  dal = q_infty*S_w*d_alpha_C_L(alpha_0,delta_E0,Q_0)
  dql = d_Q_Lift(alpha_0,delta_E0,Q_0) 
  dqm = d_Q_M_cg(alpha_0,delta_E0,Q_0)
  for eignvalue in scipy.linalg.eig(np.array([[-dal,0,1-dql],[0,0,1],[dam,0,dqm]]))[0]:
    eignvalue = np.real(eignvalue)
    if eignvalue > 0:
      stablity_of_cord['stability'].append(False)
    else:
      stablity_of_cord['stability'].append(True)

(array([    0.        +0.j,   -21.16326547+0.j, -3500.19826425+0.j]),
 array([[ 0.        ,  0.99983846, -0.01861173],
        [ 1.        , -0.00084833, -0.00028565],
        [ 0.        ,  0.01795349,  0.99982675]]))

In [29]:
print(stablity_of_cord)

{\alpha_0: [-2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -2.2818510399013454, -